In [1]:
cur_dir = os.getcwd()
os.chdir(os.path.join(cur_dir, 'data'))
!git clone https://gitlab.cs.washington.edu/ALGES/TACL2015.git
!git clone https://github.com/chaochun/nlu-asdiv-dataset.git
!git clone https://github.com/openai/grade-school-math.git
os.chdir(cur_dir)

fatal: destination path 'TACL2015' already exists and is not an empty directory.
fatal: destination path 'nlu-asdiv-dataset' already exists and is not an empty directory.
fatal: destination path 'grade-school-math' already exists and is not an empty directory.


In [1]:
import os
from xml.etree import ElementTree
import numpy as np
import torch
import json
import re
from transformers import AutoTokenizer, AutoModelForCausalLM
from io import StringIO
from contextlib import redirect_stdout
from termcolor import colored

In [2]:
import dataset_handler as dh
import helper_func as hf

In [4]:
torch.manual_seed(0)
np.random.seed(8)

gptj_model = "EleutherAI/gpt-j-6B"
codeparrot_model = "lvwerra/codeparrot"

"""Load the priming text to add to the prompt and sample a question"""
# priming_text_path = "data/priming_texts/gsm8k_onlyfullanswer.txt"
# priming_text = read_string_from_file("data/priming_texts/singleEq.txt")
priming_text_path = "data/priming_texts/asdiv/asdiv.txt"

current_dataset = dh.init_dataset_from_name("asdiv", primingtext_path=priming_text_path)

sample_q_list, sample_a_list = current_dataset.sample_n_for_prompting(10)

print(colored(sample_q_list[0], "blue"))
print(colored(sample_a_list[0], "green"))

When they got there, they saw that SFL is already filled with people. Every entrance has a long line, with 283 people waiting to get in. If SFL has 5 entrances, Write a program that prints how many people in total are waiting to get in?
1415 (people)


In [5]:
"""GPT-J and codeparrot models run in HFTest venv"""
tokenizer = AutoTokenizer.from_pretrained(gptj_model)
model = AutoModelForCausalLM.from_pretrained(gptj_model).half().eval().cuda()

In [6]:
torch.manual_seed(42)
np.random.seed(42)

n = 4
k = 3
"""n = 3
k = 3"""

pass_k_list = []

cnt = 0
for sample_q, sample_a in zip(sample_q_list, sample_a_list):
    cnt += 1
    prompt = f"{current_dataset.priming_text}\n\n#{sample_q}"
    #print(colored(f"\n\nPrompt:\n{prompt}", "green"))

    tokens = tokenizer(prompt, return_tensors="pt").input_ids
    generated_tokens = model.generate(
        tokens.long().cuda(),
        use_cache=True,
        do_sample=True,
        top_k=50,
        temperature=0.4,
        top_p=0.9,
        min_length=1,
        max_length=len(tokens[0]) + 100,
        num_return_sequences=n,
        pad_token_id=tokenizer.eos_token_id,
    )

    list_outputs = hf.preproc_gen_toks(generated_tokens, len(tokens[0]), tokenizer)

    is_correct_list = [current_dataset.verify_pred_from_output(output, sample_q, sample_a) for output in list_outputs]

    c = is_correct_list.count(True)

    pass_k = hf.pass_at_k(n, c, k)
    pass_k_list.append(pass_k)
    
    if cnt % 10 == 0:
        print(colored(f"@sample {cnt} -> Pass@{k} = {np.mean(np.array(pass_k_list))}", "green"))
        

@sample 10 -> Pass@3 = 0.475


In [7]:
print(f"Pass@{k} = {np.mean(np.array(pass_k_list))}")

Pass@3 = 0.475
